In [ ]:
import torch
from utils import (
    create_conductivity_field,
    sine_cosine_source,
)
from IPython.display import clear_output
from utils import SimpleSigma, compare

In [ ]:
device = "cpu"

In [ ]:
from matplotlib import pyplot as plt
from tqdm import tqdm

from inverse_solver import estimate_conductivity
from heat_solver import heat_steps
from utils import boundary_mask, precompute_source_history


def experiment(
    sigma_gt,
    source_func,
    noise_level=0.05,
    max_iters=10,
    alpha=1e-4,
    lr=1e-2,
    T=1.0,
    sigma_0=1.0,
    max_sigma=5.0,
    device: str = "cpu",
):
    M = sigma_gt.shape[0]
    assert sigma_gt.shape[0] == sigma_gt.shape[1]

    # Steppings
    h = 1.0 / M
    tau = h**2 / (4 * max_sigma)
    n_fwd_steps = int(T / tau) + 1
    tau = T / n_fwd_steps

    # Precalculate evolution
    source_history = precompute_source_history(source_func, M, n_fwd_steps, tau, device)
    u_gt_history = heat_steps(
        torch.zeros(sigma_gt.shape, device=device),
        sigma_gt,
        source_history,
        h,
        tau,
    )

    # Select boundary
    u_gt_b_history = u_gt_history[:, boundary_mask(M, device)]

    # Apply noise
    u_gt_b_noisy_history = (
        1 + noise_level * torch.randn_like(u_gt_b_history)
    ) * u_gt_b_history

    # Iterate inverse problem solver
    fig, ax = plt.subplots(1, 1)
    conductivity_field = SimpleSigma(M, sigma_0).to(device)
    data_loss_history = []
    reg_loss_history = []
    total_loss_history = []
    try:
        for data_term, reg_term, total_loss in tqdm(
            estimate_conductivity(
                conductivity_field,
                u_gt_b_noisy_history,
                source_history,
                sigma_0,
                alpha,
                h,
                tau,
                lr,
                max_iters,
            ),
            total=max_iters,
        ):
            data_loss_history.append(data_term)
            reg_loss_history.append(reg_term)
            total_loss_history.append(total_loss)

            ax.clear()
            ax.set_title("Loss")
            ax.plot(data_loss_history, label="Data loss", alpha=0.8)
            ax.plot(reg_loss_history, label="Reg loss", alpha=0.8)
            ax.plot(total_loss_history, label="Total loss", alpha=0.8)
            ax.legend()
            ax.set_yscale("log")
            clear_output(wait=True)
            display(fig)
    except KeyboardInterrupt:
        pass

    plt.close(fig)

    res_sigma = conductivity_field()

    print("Building gt vs obtained conductivity...")
    sigma_gt_vis = torch.as_tensor(sigma_gt).detach().cpu()
    res_sigma_vis = torch.as_tensor(res_sigma).detach().cpu()
    fig_sigma, axes = plt.subplots(1, 3, figsize=(12, 4))
    for ax, data, title in zip(
        axes,
        (sigma_gt_vis, res_sigma_vis, torch.abs(res_sigma_vis - sigma_gt_vis)),
        ("Ground truth sigma", "Recovered sigma", "Absolute error"),
    ):
        im = ax.imshow(data, cmap="viridis", origin="lower")
        ax.set_title(title)
        ax.set_xticks([])
        ax.set_yticks([])
        fig_sigma.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    display(fig_sigma)
    plt.close(fig_sigma)

    print("Building affect on temperature evolution...")
    u_history = heat_steps(
        torch.zeros_like(res_sigma, device=device),
        res_sigma,
        source_history,
        h,
        tau,
    )
    u_history = torch.cat(
        [torch.zeros_like(res_sigma, device=device).unsqueeze(0), u_history[:-1]], dim=0
    )
    u_gt_history = torch.cat(
        [torch.zeros_like(res_sigma, device=device).unsqueeze(0), u_gt_history[:-1]],
        dim=0,
    )
    compare(u_history, u_gt_history, source_history, tau)

In [ ]:
sincos10_source = lambda x, y, t: (
    (1.0 if t < 0.5 else 0.0) * sine_cosine_source(x, y, t, omega=0.1, device=device)
)
sigma_gt = 0.1 * create_conductivity_field(pattern="linear", device=device)
experiment(sigma_gt, sincos10_source, device=device, max_iters=100)